In [0]:
# If notebook is in the repo root, and dashboard is in dashboards/
import os, json, base64, requests

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

In [0]:


# Read all .lvdash.json files from local workspace repo
dashboard_folder = "./dashboards"
for fname in os.listdir(dashboard_folder):
    if fname.endswith(".lvdash.json"):
        dashboard_name = fname.replace(".lvdash.json", "")
        target_path = f"/Users/{user}/Databricks-WAF-Auto/dashboards/{dashboard_name}"

        with open(os.path.join(dashboard_folder, fname), "r", encoding="utf-8") as f:
            content = f.read()

        encoded = base64.b64encode(content.encode("utf-8")).decode("utf-8")

        response = requests.post(
            url=f"{api_url}/api/2.0/workspace/import",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json={
                "path": target_path,
                "format": "SOURCE",
                "language": "SQL",
                "overwrite": True,
                "content": encoded
            }
        )

        if response.status_code == 200:
            print(f"✅ Imported: {target_path}")
        else:
            print(f"❌ Failed: {target_path} — {response.status_code}")
            print(response.text[:300])
